## Studying the MCMC with $L^1$ regularization

#### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from implementations.mcmc_l1 import sample

In [2]:
RANDOM_SEED = 2022
rng = np.random.default_rng(RANDOM_SEED)

In [3]:
import arviz as az
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

#### Loading the data

In [4]:
f = open("./data/spambase.columns", "r")
headers = [line.split(":")[0] for line in f]+["class"]

df = pd.read_csv("./data/spambase.data", header=None) 
df.columns = headers
df.insert(0, "intercept", 1)
df["class"] = df["class"].replace(0,-1)

df.sample(5)

,intercept,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,class
2533,1,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.000,0.0,0.000,0.000,0.0,4.791,71,115,-1
4235,1,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.478,0.0,0.000,0.000,0.0,1.333,4,28,-1
471,1,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,...,0.0,0.000,0.0,1.081,0.000,0.0,2.300,9,23,1
1794,1,0.15,0.15,0.31,0.0,0.15,0.0,0.46,0.0,0.0,...,0.0,0.092,0.0,0.123,0.000,0.0,6.268,196,608,1
212,1,0.00,0.00,0.00,0.0,0.00,0.0,1.33,0.0,0.0,...,0.0,0.000,0.0,0.202,1.417,0.0,29.125,223,233,1


In [5]:
# Compute X by multiplying each row by the class
X = np.array(df.drop('class', axis=1)) * np.array(df["class"]).reshape(-1,1)

In [6]:
# Number of samples and of dimensions
n, k = X.shape
n, k

(4601, 58)

In [7]:
# Compute variance matrix
sigma = df.drop('class', axis=1).std(axis=0)
sigma[0] = 1
sigma.shape

(58,)

#### Sampling

In [8]:
n_chains = 2 # number of chains
T = 2000 # number of iterations
n_start = 200 # burn in

We now draw samples with our `sample` function, that implements the authors' algorithm from section 4.1 of the paper.

In [ ]:
nu = 1
betas1 = np.zeros((n_chains,T,k))
for i in range(n_chains):
    betas1[i] = sample(X, sigma, rng, nu, T)

nu = 1e-3
betas2 = np.zeros((n_chains,T,k))
for i in range(n_chains):
    betas2[i] = sample(X, sigma, rng, nu, T)

####  Sparsity of the coefficients

For each coefficient of $\beta$, we chose to average the results from all chains and for all time steps after a burn-in period of length `n_start`. We then plot the standardized value of $\beta$, as advised in the paper.

In [ ]:
betas1_mean_chain = betas1.mean(axis=0)/n_chains
std_betas1 = betas1_mean_chain[n_start:].mean(axis=0)*sigma

betas2_mean_chain = betas2.mean(axis=0)/n_chains
std_betas2 = betas2_mean_chain[n_start:].mean(axis=0)*sigma

We can also compute an empirical quantile interval for $\beta$, at level 95%.

In [ ]:
lower_bound1 = np.quantile(betas1_mean_chain, 0.025, axis = 0)
lower_bound2 = np.quantile(betas2_mean_chain, 0.025, axis = 0)

upper_bound1 = np.quantile(betas1_mean_chain, 1-0.025, axis = 0)
upper_bound2 = np.quantile(betas2_mean_chain, 1-0.025, axis = 0)

err1 = upper_bound1-lower_bound1
err2 = upper_bound2-lower_bound2

We summarize the results in the following plot:

In [ ]:
x = np.arange(k)
width = 0.35

fig, ax = plt.subplots(figsize=(16, 6))
bar1 = ax.bar(x - width / 2, std_betas1, width, label='$ν = 1$', alpha=.7, yerr=err1, capsize=2, color="cyan")
bar2 = ax.bar(x + width / 2, std_betas2, width, label='$ν = 10^{-3}$', alpha=.7, yerr=err2, capsize=2, color="orange")

ax.set_xticks(x)
ticks = df.columns[:-1]
ax.set_xticklabels(ticks, fontsize=8)

plt.xticks(rotation=90)
#plt.yscale("log")
plt.legend()
plt.show()

#### Convergence of the chains

In [ ]:
n_coeffs = 2
coeffs = np.random.randint(k, size = n_coeffs)
colors = ["blue", "red", "green"]

In [ ]:
print('For ν = 1')
for coeff in coeffs:
    for i in range(n_chains):
        trajectory = betas1[i,n_start:,coeff]
        plt.plot(trajectory, color = colors[i], alpha = 0.5)
        plt.axhline(y=np.mean(trajectory), linestyle="--", color = colors[i])
    plt.title(ticks[coeff])
    plt.show()

In [ ]:
print('For ν = 10^{-3}')
for coeff in coeffs:
    for i in range(n_chains):
        trajectory = betas2[i,n_start:,coeff]
        plt.plot(trajectory, color = colors[i], alpha = 0.5)
        plt.axhline(y=np.mean(trajectory), linestyle="--", color = colors[i])
    plt.title(ticks[coeff])
    plt.show()

We can observe that the chains seem to mix pretty well according to the plots.  
Running 10 000 iterations as the authors do would probaby be safer, but unfortunately our computers can't handle that amount of computations.